<a href="https://colab.research.google.com/github/camerontruong/ARA-READABILITY/blob/main/READABILITY_AWARDS_V3_07JUNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install textstat
!pip install PyPDF2
!pip install -U textblob
import os
import glob
import textstat
import csv
import PyPDF2
from textblob import TextBlob

Requirement already up-to-date: textblob in /usr/local/lib/python3.7/dist-packages (0.15.3)


In [ ]:
TARGET_FILES = r'/content/drive/My Drive/Control sample/*.pdf'
file_list = glob.glob(TARGET_FILES)

In [ ]:
new_list=[]
name=[]
size=[]
pagen=[]
for file in file_list:
  print(file)
  n=(file.split('.pdf',0))
  name.append(n)
  size.append((os.stat(file).st_size)/1000000)
    
  
  if file.endswith(".pdf"):
    with open (file,'rb') as pdfobject:
      pdfreader=PyPDF2.PdfFileReader(pdfobject)
      page_numbers=pdfreader.numPages
      pagen.append(page_numbers)
      if pdfreader.isEncrypted == True:
        pass
      else:
          currentpage = 0
          text = ""
          while (currentpage <page_numbers):
            page=pdfreader.getPage(currentpage)
            try:
              text=text+page.extractText()
            except:
                print(currentpage) #to print the error page
            currentpage +=1
          new_list.append(text)  
          


In [ ]:
gf=[]
kg=[]
ke=[]
dc=[]
sc=[]
dif=[]
st=[]
ct=[]
sep=[]
ses=[]
for test in new_list:
  #print(textstat.gunning_fog(test))
  #print(textstat.lexicon_count(test, removepunct=True))
  gf.append(textstat.gunning_fog(test))
  kg.append(textstat.flesch_kincaid_grade(test))
  ke.append(textstat.flesch_reading_ease(test))
  dc.append(textstat.dale_chall_readability_score(test))
  sc.append(textstat.sentence_count(test))
  dif.append(textstat.difficult_words(test))
  st.append(textstat.text_standard(test))
  ct.append(textstat.lexicon_count(test, removepunct=True))
  sep.append(TextBlob(test).sentiment.polarity)
  ses.append(TextBlob(test).sentiment.subjectivity)

In [ ]:
ticker=[]
year=[]
for file in file_list:
  ticker.append((file.rsplit('/content/drive/My Drive/Control sample/')[1][0:3]))
  year.append((file.rsplit('/content/drive/My Drive/Control sample/')[1][4:8]))

In [ ]:
from itertools import zip_longest
data = [ticker, year, gf, kg,ke,dc,sc,dif,st,name,ct, pagen, size,sep,ses]
export_data = zip_longest(*data, fillvalue = '')
with open('/content/drive/My Drive/MSFT/READABILITY_CONTROLS.csv', 'w', encoding="ISO-8859-1", newline='') as file:
      write = csv.writer(file)
      write.writerow(("TICKER", "YEAR", "GUNNING FOG", "FLESCH-KINCAID GRADE","FLESCH-READING EASE","DALE-CHALL","SENTENCE COUNT","DIFFICULT WORDS","STANDARD READABILITY","NAME","TOTAL WORDS","TOTAL PAGES","FILE SIZE","SENTIMENT POLARITY","SENTIMENT SUBJECTIVITY"))
      write.writerows(export_data)